<a href="https://colab.research.google.com/github/Varun-dasari/INFO5731_Spring2020/blob/master/Copy_of_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [0]:
#Joker Reviews
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup as BS
import re
import numpy as np
import pandas as pd

def get_tablelist(soup):
    reviews = soup.find_all("div", {"class": "lister-item-content"})

    review_data = []

    for review in reviews:

        try:
            review_content = review.find("div", {"class": "text show-more__control"}).text.strip()
        except:
            review_content = review.find("div", {"class": "text show-more__control clickable"}).text.strip()

        review_data.append(review_content)
    return review_data

b_url = 'https://www.imdb.com/'
url = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
res = requests.get(url)
soup = BS(res.text, 'html.parser')
res.encoding = 'utf-8'
allTablelist=[]
tablelist = get_tablelist(soup)
for i in tablelist:
  allTablelist.append(i)

load_more = soup.select(".load-more-data")
flag = True
if len(load_more):
    ajaxurl = load_more[0]['data-ajaxurl']
    b_url = b_url + ajaxurl + "?ref_=undefined&paginationKey="
    try:
        key = load_more[0]['data-key']
    except KeyError:
        flag = False
else:
    flag = False

while flag:
    url = b_url + key
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BS(res.text, 'html.parser')
    tablelist2 = get_tablelist(soup)
    if len(allTablelist) == 100:
      break
    for i in tablelist2:
        allTablelist.append(i)
    load_more = soup.select(".load-more-data")
    if len(load_more):
        key = load_more[0]['data-key']
    else:
        flag = False

data = np.array(allTablelist)
data = pd.DataFrame(data, columns=['review_content'])
data.shape
data.to_csv('Reviews.csv')
data



,review_content
0,I was a person that saw all the hype and claim...
1,"Every once in a while a movie comes, that trul..."
2,This is a movie that only those who have felt ...
3,"Truly a masterpiece, The Best Hollywood film o..."
4,Most of the time movies are anticipated like t...
...,...
95,"Dark, depressing and unsettling film with a ha..."
96,I just didn't get the hype about this movie. A...
97,Story was just really unconvincing. Nobody rea...
98,JOKER is a gift to the audiences. I felt as a ...


In [0]:
#Calculating the Frequency N grams where N = 3 (Trigrams)
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
d = pd.read_csv('Reviews.csv', low_memory=False)
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(data['review_content'].values.astype('U')) 
frequencies = sum(sparse_matrix).toarray()[0]
df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
df

,Frequency
10 10 amazing,1
10 10 edit,1
10 10 hope,1
10 10 oscar,1
10 10 reviews,1
...,...
zazie beetz was,1
zazie yas potential,1
zero nuance subtlety,1
zero recoil and,1


In [0]:
import csv
with open('Reviews.csv') as f:
 reader = csv.reader(f)
 next(reader, None)
 review_content = [row[1] for row in reader]
with open('review_content.txt', mode="w") as outfile:
    for s in review_content:
        outfile.write("%s\n" % s)

#Bigrams
#Calculating probabilities of bi-grams 
def bigramEstimation(file):
    lst = []
    unigrams = {}
    bigrams = {} 
    text = open(file, 'r').read()
    lst = text.strip().split()
    del text 
    for l in lst:
        if not l in unigrams:
            unigrams[l] = 1
        else:
            unigrams[l] += 1
    for i in range(len(lst) - 1):
        temp = (lst[i], lst[i+1])
        if not temp in bigrams:
            bigrams[temp] = 1
        else:
          bigrams[temp] += 1
    print('Generated ', len(bigrams), ' bigrams')
    total_corpus = sum(unigrams.values())
    for k,v in bigrams.items():
        first_word = k[0]
        first_word_count = unigrams[first_word]
        bi_prob = bigrams[k] / unigrams[first_word]
        if(v == 2):
         print(k[0] ,k[1],v ,bi_prob)

In [0]:
#Calculating the Probability of Nouns
import spacy
import csv
from tabulate import tabulate


nlp = spacy.load("en")
f = open("review_content.txt", "r")
doc = nlp(f.read())
noun_phrases = []
for np in doc.noun_chunks:
  noun_phrases.append(np.text)
print(noun_phrases)
dfn = pd.DataFrame(noun_phrases, columns = ['noun_phrases'])
word_vectorizer = CountVectorizer(ngram_range=(2,7), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(dfn['noun_phrases'].values.astype('U')) 
frequencies = sum(sparse_matrix).toarray()[0]
dff = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
dff['Probability'] = dff['Frequency'] / dff['Frequency'].max()
result = dff.astype(object).transpose() 
result

['I', 'a person', 'all the hype', 'claims', 'masterpiece', 'overreacting and overblown excitement', 'another Joker', 'film', 'I', 'the trailer', 'I', 'a massive achievement', 'cinema', 'a day', 'age', 'cgi nonsense', 'reboots', 'a reboot', 'sorts', 'the standalone origin tale', 'start', 'resemblance', 'the best joker', 'origin', 'comics', 'the past', 'Joaquin', 'sweats', 'his every drop', 'this magnificently dedicated performance', 'Heath Ledger', 'the greatest acting performance', "Heath's joker", 'The directing', 'writing', 'the bleak settings', 'tones', 'this film', 'the place', 'every audience member', 'they', 'a film', 'them', "a character's world", 'very existence', 'the hype', 'a transcending masterpiece', 'cinema', 'a while', 'a movie', 'an impact', "Joaquin's performance", 'scenography', 'it', 'brilliance', 'Grotesque', 'haunting', 'cringy', 'times', 'you', 'an eye', 'it', 'seriously funny moments', 'Emotional rollercoaster', 'multiple emotions', 'the same time.this', 'a typic

,10 10,10 10 amazing,10 10 amazing story,10 10 oscar,10 10 oscar worthy,10 10 oscar worthy performanceby,10 10 reviews,10 10 the,10 10 the reason,10 amazing,10 amazing story,10 oscar,10 oscar worthy,10 oscar worthy performanceby,10 points,10 rating,10 reviews,10 score,10 star,10 star reviews,10 the,10 the reason,1000 stake,12y olds,1970s gotham,1970s gotham city,20 years,37 seconds,45 minutes,70 shock,70 shock horror,70 shock horror flicks,70s psycho,70s psycho horror,70s psycho horror vibe,76th venice,76th venice international,76th venice international film,76th venice international film festival,80 min,...,yet dead serious,yet dead serious speech,young boy,young bruce,young bruce wayne,younger audience,your bones,your darkest,your darkest secrets,your deepest,your deepest thoughts,your entire,your entire review,your friends,your grade,your grade school,your grade school playground,your hand,your idea,your kids,your life,your opinion,your seat,your skinny,your skinny lonely,your skinny lonely drugged,your skinny lonely drugged up,your skinny lonely drugged up psychos,your superhero,your superhero movie,your thoughts,your typical,your typical super,your typical super hero,your typical super hero movie,zazie beetz,zazie yas,zero nuance,zero recoil,zero understanding
Frequency,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Probability,0.0380952,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.0190476,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,...,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.0190476,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381,0.00952381


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
#TDIDF Document
df = pd.read_csv("Reviews.csv")
tf2 = df.dropna()
tf1 = (tf2['review_content'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index())
tf1.columns = ['Word','Term_Frequency']
tf1


,Word,Term_Frequency
0,and,492.0
1,the,717.0
2,a,481.0
3,is,330.0
4,of,374.0
...,...,...
4037,earlier,1.0
4038,better.,1.0
4039,homage,1.0
4040,placed,1.0


In [0]:
#Cosine similarity for all the documents
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
doc_set = tf2['review_content'].values.tolist()
query_set = "This Is without a doubt the best protryal of the joker, the way he slowly gets turned by society and becomes skinnier and crazier,defenanrly one of the if not the best movie of 2019"
query_set = [query_set]
stopWords = stopwords.words('english')
vectorizer = CountVectorizer(stop_words = stopWords)
transformer = TfidfTransformer()
docVectorizerArray = vectorizer.fit_transform(doc_set).toarray()
queryVectorizerArray = vectorizer.transform(query_set).toarray()
cx = lambda a, b : np.inner(a, b)/(LA.norm(a)*LA.norm(b))
result = []
for vector in docVectorizerArray:
        for testV in queryVectorizerArray:
            cosine = cx(vector, testV)
            result.append(cosine)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#Cos Similarity Based Ranking
n = tf2.filter(['Unnamed','review_content'], axis=1)
se = pd.Series(result)
n['Cos-similarity'] = se.values
n.drop(n.loc[n['Cos-similarity']==0].index, inplace=True)
n["Rank"] = n["Cos-similarity"].rank().astype(int)
n.sort_values("Cos-similarity", inplace = True) 
n

,review_content,Cos-similarity,Rank
27,When I heard everyone saying that this is the ...,0.023783,1
92,To say it was satisfying would be somewhat mis...,0.026948,2
99,There is doing an homage and then there is bor...,0.031054,3
56,Don't waste 2 hours of your life this movie is...,0.039873,4
44,Yes Joaquin's performance is good and the cine...,0.041595,5
...,...,...,...
38,"Seriously, unbelievable. Dare I say better tha...",0.270226,87
71,It seems like they were making a movie about p...,0.275041,88
41,While the trailers for this movie have may see...,0.288043,89
73,Best acting and story. This is an Oscar winnin...,0.297044,90


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file
https://github.com/Varun-dasari/SampleData/blob/master/SentimetalAnalysis.csv